In [28]:
from sklearn.datasets import fetch_openml #download Minist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.naive_bayes import MultinomialNB

In [12]:
len(set(y_train))

10

In [20]:
class MyNavieBayes:

    def __init__(self, X_train, y_train, X_test, y_test) -> None:
        '''
        载入数据集, 需要传入ndarray类型     
        '''
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.featureNum = X_train.shape[1] #训练数据特征数
        self.classNum = len(set(y_train))  #训练数据类数
    
    def NaiveBayes(self, lambda_smooth=1):
        '''
        NaiveBayes需要学习的参数:
            P(Y): Prior -> sum(y_i)/total(Y)
            P(X|Y): Likelihood -> 
        params:
            X_train: 训练数据集
            y_train: 训练标签
        Note: 拉普拉斯平滑 lambda = 1
        '''
        y_train = self.y_train
        X_train = self.X_train
        # 1. 计算先验概率P(Yi)：每个Y_i的P(Y_i)
        p_y = np.zeros((self.classNum, 1))
        for i in range(self.classNum):
            p_y[i] = ((np.sum(y_train == i)) + lambda_smooth) / (len(y_train) + self.classNum*lambda_smooth)
        # 2. 计算似然P(Xj|Yi): 
        p_x_y = np.zeros((self.classNum, self.featureNum))
        for i in range(self.classNum):
            p_x_y[i,:] = (np.sum(X_train[y_train == i,:], axis=0) + 1)/(np.sum(y_train == i) + 2)
        
        return p_x_y, p_y
    
    def predict(self, p_y, p_x_y):
        X_test = self.X_test
        num_test = self.X_test.shape[0]
        # 计算后验概率 P(Y|X)
        # 为了避免累乘造成的下溢，我们用log转换成sum
        log_p_y = np.log(p_y)
        log_p_x_y = np.log(p_x_y)
        p_y_x = np.zeros((num_test, self.classNum))
        for i in range(self.classNum):
            p_y_x[:, i] = log_p_y[i] + np.sum(X_test*log_p_x_y[i, :] + 
                                              (1-X_test)*np.log(1-np.exp(log_p_x_y[i,:])), axis=1)
            
            
        return np.argmax(p_y_x, axis=1)
        

In [35]:
# mnist = fetch_openml('mnist_784')
X = mnist.data/255.0
y = mnist.target.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [36]:
bayesclf = MyNavieBayes(X_train=X_train,
                        y_train=y_train,
                        X_test = X_test,
                        y_test=y_test
                        )
                        

In [37]:
p_x_y, p_y = bayesclf.NaiveBayes()
y_pred = bayesclf.predict(p_y = p_y, p_x_y=p_x_y)

In [38]:
accuracy_score(y_pred, bayesclf.y_test)

0.838

In [39]:
# sklearn库的Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_pred, y_test)

0.8288571428571428